<a href="https://colab.research.google.com/github/nvvishnu/CS6910-Assignments/blob/main/Assignment2/partB/Assignment2_partB_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras import *
!pip install wandb
import wandb
from wandb.keras import WandbCallback 
import math

In [ ]:
def train_model(x,retrain_top_layers_percentage,augmentation,epochs,target_size,optimizer):
    
    g = len(x.layers)
    print('layers_size: ',g)
    num_retrain = math.ceil(g*(1-retrain_top_layers_percentage))
    for gg in x.layers[:num_retrain]:
      gg.trainable = False
    print(num_retrain)
    model = Sequential()
    model.add(x)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10))
    model.add(layers.Softmax())
    model.summary()
    datagen = preprocessing.image.ImageDataGenerator()
    path = '/content/drive/MyDrive/Assignment2/inaturalist_12K/'
    augmented = preprocessing.image.ImageDataGenerator(
                              #featurewise_center=True,
                              #featurewise_std_normalization=True,
                              rotation_range=20,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True)
    
    if(augmentation):
      train = augmented.flow_from_directory(path+'split/train/',target_size=target_size,color_mode='rgb', classes=None,class_mode='categorical', batch_size=32)
    else:
       train = datagen.flow_from_directory(path+'split/train/',target_size=target_size,color_mode='rgb', classes=None,class_mode='categorical', batch_size=32)
    validation = datagen.flow_from_directory(path+'split/val/',target_size=target_size,color_mode='rgb', classes=None,class_mode='categorical', batch_size=32)
     #x is input model
    model.compile(optimizer=optimizer,loss=losses.CategoricalCrossentropy(),metrics=['accuracy'])
    model.fit(train,epochs=epochs,validation_data=validation,callbacks=[WandbCallback()])

    return model

def fun(model_name,retrain_top_layers_percentage,augmentation,epochs,optimizer):
    
    if(model_name == 'NASNetLarge'):
       x = applications.NASNetLarge(include_top=False, weights='imagenet',input_shape=(331,331,3))
       target_size = (331,331)
    elif(model_name == 'InceptionV3'):
       x = applications.InceptionV3(include_top=False, weights='imagenet',input_shape=(224,224,3))
       target_size = (224,224)
    elif(model_name == 'ResNet50'):
       x = applications.ResNet50(include_top=False, weights='imagenet',input_shape=(224,224,3))
       target_size = (224,224)
    elif(model_name == 'Xception'):
       x = applications.Xception(include_top=False, weights='imagenet',input_shape=(224,224,3))
       target_size = (224,224)
    elif(model_name == 'InceptionResNetV2'):
       x = applications.InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(224,224,3))
       target_size = (224,224)
    elif(model_name == 'ResNet101'):
      x = applications.ResNet101(include_top=False,weights='imagenet',input_shape=(224,224,3))
      target_size = (224,224)
    elif(model_name == 'ResNet50V2'):
      x = applications.ResNet50V2(include_top=False,weights='imagenet',input_shape=(224,224,3))
      target_size = (224,224)
    elif(model_name == 'VGG16'):
      x = applications.VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))
      target_size = (224,224)
    else:
      raise Exception("Invalid Model name")
    
    return train_model(x,retrain_top_layers_percentage,augmentation,epochs,target_size,optimizer)
    